<a href="https://colab.research.google.com/github/guillaumetougas/sbl_hyperspectral_ai/blob/main/02_scripts/sbl_hyperspec_1D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1D-CNN model from hyperspectral data


In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import torch
import torchvision as tv
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchaudio as ta
!pip install rasterio
import rasterio as rio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 94.7 MB/s eta 0:00:00


###Load preprocessed HS data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
#Load HS data with geopandas
smoothed_data=gpd.read_file("/content/drive/MyDrive/2024-Tougas-PhD/Data_SBL_HS_AI/sbl_smoothed_spectra.gpkg")

#isolate spectral data and prepare with rasterio
spectral_data_labeled = smoothed_data.drop(columns=['geometry']).values
print(np.shape(spectral_data_labeled))
print(spectral_data_labeled[:5])

spectral_data = spectral_data_labeled[:, 1:242]

(21696, 246)
[['1' 0.0014360744236254599 0.0016860739034256812 ... 'BEPA'
  20.6998377058 18.0239535584]
 ['10' 0.0 3.7468307932240064e-06 ... 'ACRU' 8.92277349821 3.50901934984]
 ['100' 0.0 0.00018155946172550302 ... 'Picea' 12.975775825 7.35715668545]
 ['1000' 0.00011515454612388588 0.000444991645046132 ... 'ACSA'
  12.9757616982 8.00540072982]
 ['10000' 0.005083690612008917 0.004094856722796195 ... 'ABBA'
  9.26431133742 5.2001091841]]
torch.Size([21696, 241])
0         BEPA
1         ACRU
2        Picea
3         ACSA
4         ABBA
         ...  
21691     THOC
21692     THOC
21693    Picea
21694    Picea
21695     Mort
Name: Label, Length: 21696, dtype: object


### Create the 1D-CNN model

In [10]:
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Example hyperspectral data (N samples, C spectral bands)
# Replace this with your actual data
N, C = 21696, 242  # Example: 21696 pixels, 242 bands
#X = spectral_data
#Y = labels

# Normalization (recommended for pretrained generic models like 1D CNN)
spectral_data = (spectral_data - np.min(spectral_data)) / (np.max(spectral_data) - np.min(spectral_data))

# Ensure spectral_data has a numeric dtype before converting to tensor
spectral_data = spectral_data.astype(np.float32)  # or np.float64

labels = smoothed_data['Label']
print(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(spectral_data, labels, test_size=0.2, random_state=42)

# 1. Create train/val indices instead of directly splitting
dataset_size = len(spectral_data)
indices = list(range(dataset_size))
split = int(np.floor(0.2 * dataset_size))  # 20% for validation
np.random.shuffle(indices)  # Shuffle indices randomly
train_indices, val_indices = indices[split:], indices[:split]

# 2. Separate data using the indices
X_train, X_val = spectral_data[train_indices], spectral_data[val_indices]
y_train, y_val = labels.iloc[train_indices], labels.iloc[val_indices]

# 3. Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# 4. Create PyTorch tensors
X_train, X_val = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_val, dtype=torch.float32)
y_train, y_val = torch.tensor(y_train_encoded, dtype=torch.long), torch.tensor(y_val_encoded, dtype=torch.long)

# Define a 1D CNN for hyperspectral data classification
class HyperspectralCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(HyperspectralCNN, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * (input_channels // 2), 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

0         BEPA
1         ACRU
2        Picea
3         ACSA
4         ABBA
         ...  
21691     THOC
21692     THOC
21693    Picea
21694    Picea
21695     Mort
Name: Label, Length: 21696, dtype: object


###Model parameters and optimizer and loss function set up





In [15]:
# Model parameters
input_channels = C
num_classes = len(torch.unique(y_train))  # Number of unique labels

# Initialize the model, loss function, and optimizer
model = HyperspectralCNN(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


###Train the model

In [16]:
#Training loop (example)
num_epochs = 10  # You can adjust this
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# ... (rest of the code: evaluation, etc.) ...

RuntimeError: Given groups=1, weight of size [64, 242, 3], expected input[1, 17357, 241] to have 242 channels, but got 17357 channels instead

###Evaluate the model

In [ ]:
# Evaluate the model on a test dataset
# ...